# Node Timeline Statistics

In [1]:
!pip install --upgrade py2neo
from py2neo import Graph, Relationship, NodeMatcher
from string import Template
import pandas as pd 

#Login + PW for the local neo4j database
graph = Graph("bolt+ssc://h2918680.stratoserver.net:7687", auth=("sonar", "sonar2021"))

     |████████████████████████████████| 204 kB 2.5 MB/s eta 0:00:01
  Attempting uninstall: py2neo
    Found existing installation: py2neo 2021.1.1
    Uninstalling py2neo-2021.1.1:
      Successfully uninstalled py2neo-2021.1.1


In [36]:
#options (startYear = first year (below 0 doesn't work yet), steps = next step in interval)
startYear = 0
steps = 50

In [9]:
query = 'MATCH (n:Resource) \
WHERE toInteger(n.DateApproxBegin) >= 1700 \
AND toInteger(n.DateApproxBegin) <= 1701 \
RETURN count(n)' 
#RETURN n.Name as Name, n.Title as Title, n.Id as Id, n.Genre as Genre, n.DateApproxBegin as DateApproxBegin, n.DateApproxEnd as DateApproxEnd, labels(n) as Label LIMIT 10' 
result = graph.run(query).data()
dfTemp = pd.DataFrame(result)
dfTemp

,count(n)
0,9417


# Count numbers per year interval for each label

In [31]:
%%time
#PerNames (schnell: 22min 47) aber Tod muss eingearbeteitet werden, damit Personen auch bei späteren Jahren auftauchen
dfFinal2 = pd.DataFrame()
for x in range(0, 40): #80
    z = x*steps
    queryTemplate = Template('\
    MATCH (n:PerName) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN ${year} As year, Count(DISTINCT n) As PerName')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    result = graph.run(query).data()
    df2 = pd.DataFrame(result)
    dfFinal2 = dfFinal2.append(df2)

dfFinal2

CPU times: user 464 ms, sys: 62.1 ms, total: 526 ms
Wall time: 22min 47s


,year,PerName
0,0,444
0,25,28
0,50,18
0,75,12
0,100,20
...,...,...
0,1900,258748
0,1925,535663
0,1950,753336
0,1975,349022


In [34]:
%%time
#MeetName (schnell: 3min 36s)
dfFinal3 = pd.DataFrame()
for x in range(0, 40): #80
    z = x*steps
    queryTemplate = Template('\
    MATCH (n:MeetName) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN ${year} As year, Count(DISTINCT n) As MeetName')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    result = graph.run(query).data()
    df3 = pd.DataFrame(result)
    dfFinal3 = dfFinal3.append(df3)

dfFinal3

CPU times: user 347 ms, sys: 40.4 ms, total: 387 ms
Wall time: 3min 36s


,year,MeetName
0,0,339
0,25,24
0,50,1
0,175,3
0,200,6
...,...,...
0,1900,4273
0,1925,7715
0,1950,53383
0,1975,338208


In [38]:
%%time
#Resourcen (sehr langsam: 3h 4min)
dfFinal = pd.DataFrame()
for x in range(0, 42): #80
    z = x*steps
    queryTemplate = Template('\
    MATCH (n:Resource) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN ${year} As year, Count(DISTINCT n) As Resource')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    result = graph.run(query).data()
    df = pd.DataFrame(result)
    dfFinal = dfFinal.append(df)

dfFinal

CPU times: user 631 ms, sys: 308 ms, total: 939 ms
Wall time: 3h 4min 43s


,year,Resource
0,0,19928
0,50,15
0,100,5
0,150,47
0,200,13
0,250,2
0,300,13
0,350,3
0,400,8
0,450,18


In [54]:
%%time
#UniTitle (sehr schnell: 50.2 s)
dfFinal4 = pd.DataFrame()
for x in range(0, 40): #80
    z = x*steps
    queryTemplate = Template('\
    MATCH (n:UniTitle) \
    WHERE (toInteger(n.DateApproxBegin) >= toInteger(${year})) AND (toInteger(n.DateApproxBegin) < toInteger(${year2})) \
    RETURN ${year} As year, Count(DISTINCT n) As UniTitle')
    query = queryTemplate.substitute(year=startYear+z, year2=startYear+z+steps)
    result = graph.run(query).data()
    df4 = pd.DataFrame(result)
    dfFinal4 = dfFinal4.append(df4)

dfFinal4

CPU times: user 164 ms, sys: 20.1 ms, total: 184 ms
Wall time: 50.2 s


,year,UniTitle
0,800,1
0,1050,6
0,1150,2
0,1200,12
0,1250,13
0,1300,14
0,1350,8
0,1400,14
0,1450,17
0,1500,70


In [64]:
#merge the data to one dataset
dfFinalAll = pd.merge(dfFinal, dfFinal2, how='left')
dfFinalAll = pd.merge(dfFinalAll, dfFinal3, how='left')
dfFinalAll = pd.merge(dfFinalAll, dfFinal4, how='left')
dfFinalAll.fillna(0).reset_index()

,index,year,Resource,PerName,MeetName,UniTitle
0,0,0,19928,444.0,339.0,0.0
1,1,50,15,18.0,1.0,0.0
2,2,100,5,20.0,0.0,0.0
3,3,150,47,31.0,0.0,0.0
4,4,200,13,34.0,6.0,0.0
5,5,250,2,34.0,1.0,0.0
6,6,300,13,52.0,1.0,0.0
7,7,350,3,46.0,3.0,0.0
8,8,400,8,53.0,1.0,0.0
9,9,450,18,57.0,2.0,0.0


In [66]:
#save as csv
dfFinalAll.to_csv("data-results/timeline-data.csv")